<img src="images/header.png">

# Алгоритмы интеллектуальной обработки больших объемов данных

## Лекция 3. Классификация текстов и Naive Bayes

In [1]:
import warnings
warnings.filterwarnings('ignore')

## План занятия
* Обработка текстов
* Naive Bayes

## Нужно отметиться на лекции

### Data Mining vs Text Mining

Data Mining:   
* извлечение *неочевидной* информации

Text Mining:  
* извлечение *очевидной* информации

Трудности  
* Огромные объемы
* Отстутсвие структуры
	    

## Задачи Text Mining
* Суммаризация текста  
агрегация новостей
* Классификация и кластеризация документов  
категоризация, антиспам, sentiment analysis, opinion mining
* Извлечение метаданных  
определение языка, автора, тегирование
* Выделение сущностей  
места, люди, компании, почтовые адреса


* автоматический перевод
* чат-бот
* поиск точных ответов на вопросы в тексте

## Этапы (простой) обработки текста

<img src="images/textm.png">


## Декодирование


**Def.**  
перевод последовательности байт в последовательность символов

* Распаковка  
*plain/.zip/.gz/...*
* Кодировка  
*ASCII/utf-8/Windows-1251/...*
* Формат  
*csv/xml/json/doc...*

Кроме того: что такое документ?



## Разбиение на токены
**Def.**  
разбиение последовательности символов на части (токены), возможно, исключая из рассмотрения некоторые символы  
Наивный подход: разделить строку пробелами и выкинуть знаки препинания  


*Трисия любила Нью-Йорк, поскольку любовь к Нью-Йорку могла положительно повлиять на ее карьеру.*  


**Проблемы:**  
* n.anokhin@corp.mail.ru, 127.0.0.1
* С++, C#
* York University vs New York University
* Зависимость от языка (“Lebensversicherungsgesellschaftsangestellter”, “l’amour”)
Альтернатива: n-граммы

## Разбиение на токены


In [8]:
# import nltk
# nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\папик\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [5]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer('\w+|[^\w\s]+')
s = u'Трисия любила Нью-Йорк, поскольку любовь к Нью-Йорку могла положительно повлиять на ее карьеру.'
for t in tokenizer.tokenize(s): 
    print(t)

Трисия
любила
Нью
-
Йорк
,
поскольку
любовь
к
Нью
-
Йорку
могла
положительно
повлиять
на
ее
карьеру
.


## Стоп-слова
**Def.**  
Наиболее частые слова в языке, не содержащие никакой информации о содержании текста



In [9]:
from nltk.corpus import stopwords
print(' '.join(stopwords.words('russian')[1:20]))

в во не что он на я с со как а то все она так его но да ты


Проблема: “To be or not to be"

## Нормализация
**Def.**  
Приведение токенов к единому виду для того, чтобы избавиться от поверхностной разницы в написании  

Подходы  
* сформулировать набор правил, по которым преобразуется токен  
Нью-Йорк → нью-йорк → ньюйорк → ньюиорк
* явно хранить связи между токенами (WordNet – Princeton)  
машина → автомобиль, Windows 6→ window

In [11]:
s = u'Нью-Йорк'
s1 = s.lower()
print(s1)

нью-йорк


In [13]:
import re
s2 = re.sub(r"\W", "", s1, flags=re.U)
print(s2)

ньюйорк


In [14]:
s3 = re.sub(r"й", u"и", s2, flags=re.U)
print(s3)

ньюиорк


## Стемминг и Лемматизация
**Def.**  
Приведение грамматических форм слова и однокоренных слов к единой основе (lemma):
* Stemming – с помощью простых эвристических правил
  * Porter (Cambridge – 1980)
        5 этапов, на каждом применяется набор правил, таких как
            sses → ss (caresses → caress)
            ies → i (ponies → poni)

  * Lovins (1968)
  * Paice (1990)
  * другие
* Lemmatization – с использованием словарей и морфологического анализа


## Стемминг

In [16]:
from nltk.stem.snowball import PorterStemmer
s = PorterStemmer()
print(s.stem('Tokenization'))
print(s.stem('stemming'))

from nltk.stem.snowball import RussianStemmer
r = RussianStemmer()
print(r.stem(u'Авиация'))
print(r.stem(u'национальный'))

token
stem
авиац
национальн


**Наблюдение**  
для сложных языков лучше подходит лемматизация

## Лемматизация

In [ ]:
import pymorphy2

import pymorphy2
morph = pymorphy2.MorphAnalyzer()
print(morph.parse(u'думающему')[0].normal_form)

## Heaps' law

$$
M = k T^\beta, \;M \text{ -- размер словаря}, \; T \text{ -- количество слов в корпусе}
$$
$$
30 \leq k \leq 100, \; b \approx 0.5
$$

<img src="images/dim.png">
<img src="images/heaps.png">

## Представление документов
**Boolean Model.** Присутствие или отсутствие слова в документе  
**Bag of Words.** Порядок токенов не важен  

*Погода была ужасная, принцесса была прекрасная.
Или все было наоборот?*

Координаты
* Мультиномиальные: количество токенов в документе
* Числовые: взвешенное количество токенов в документе

## Zipf's law

$t_1, \ldots, t_N$ - токены, отранжированные по убыванию частоты
   	
$f_1, \dots, f_N$ - соответствующие частоты

**Закон Ципфа**
	$$
	f_i = \frac{c}{i^k}
	$$	
	
	Что еще? Посещаемость сайтов, количество друзей, население городов...
<img src="images/zipf.png">


## TF-IDF

Количество вхождений слова $t$ в документе $d$
$$
TF_{t,d} = term\!\!-\!\!frequency(t, d)
$$
Количество документов из $N$ возможных, где встречается $t$
$$
DF_t = document\!\!-\!\!fequency(t)
$$
$$
IDF_t = inverse\!\!-\!\!document\!\!-\!\!frequency(t) = \log \frac{N}{DF_t}
$$
TF-IDF
$$
TF\!\!-\!\!IDF_{t,d} = TF_{t,d} \times IDF_t
$$

Оценивает важность слова в контексте документа, являющегося частью корпуса


## Перерыв
## Опрос

## Байесовский классификатор

Дано

$\mathbf{x} \in X$ - описание документа $d$ из коллекции $D$  
$C_k \in C, \; k = 1,\ldots,K$ - целевая переменная

Теорема Байеса
$$
P(C_k \mid \mathbf{x}) = \frac{p(\mathbf{x} \mid C_k) p(C_k)}{p(\mathbf{x})} \propto p(\mathbf{x} \mid C_k) p(C_k)
$$

Принцип Maximum A-Posteriori
$$
C_{MAP} = \arg \max_k p(C_k | \mathbf{x})
$$

Байесовский классификатор — широкий класс алгоритмов классификации, основанный на принципе максимума апостериорной вероятности.  
Для классифицируемого объекта вычисляются функции правдоподобия каждого из классов, по ним вычисляются апостериорные вероятности классов.  
Объект относится к тому классу, для которого апостериорная вероятность максимальна.


## Naive Bayes

$x_j$ - слово на $j$-м месте в документе $\mathbf{x}$,  
$w^i \in V$ - слово из словаря $V$


Предположения
* conditional independence - слова внутри документа независимы
$$
p(x_i=w^s, x_j=w^r | C_k) = p(x_i=w^s | C_k) p(x_j=w^r | C_k)
$$
* postional independence - результат не зависит от позиции слова в документе
$$
P(x_i=w^s | C_k) = P(x_j=w^s | C_k) = P(x = w^s | C_k)
$$

Получаем
$$
p(\mathbf{x} | C_k) = p(x_1=w^{s_1}, \ldots, x_{|\mathbf{x}|}=w^{s_{|\mathbf{x}|}} | C_k) = \prod_{i=1}^{|\mathbf{x}|} p(x = w^{s_i} | C_k)
$$

**Почему NB хорошо работает?**  
Корректная оценка дает правильное предсказание, но правильное предсказание *не требует* корректной оценки


## Варианты NB

MAP
$$
C_{MAP} = \arg \max_k p(C_k) \prod_{i=1}^{|\mathbf{x}|} p(x = w^{s_i} | C_k)  = 
$$
$$
= \arg \max_k \left[ \log p(C_k) + \sum_{i=1}^{|\mathbf{x}|} \log p(x = w^{s_i} | C_k) \right]
$$
Априорные вероятности
$$
p(C_k) = N_{C_k}/{N}
$$
Likelihood $p(x = w^{s_i} | C_k)$
* BernoulliNB $p(x = w^{s_i} | C_k) = D_{w^{s_i}, C_k} / D_{C_k}$, $D$ - кол-во документов
* MultinomialNB $p(x = w^{s_i} | C_k) = T_{w^{s_i}, C_k} / T_{C_k}$, $T$ - кол-во токенов
* GaussianNB $p(x = w^{s_i} | C_k) = \mathcal{N}(\mu_k, \sigma_k^2)$, параметры из MLE


## Обучение NB

```
function nb_train(D,C):
	V = dictionary of tokens
	N = number of documents
	for Ck in C: # iterate over all classes
		N_Ck = number of documents in class Ck
		p(Ck) = N_Ck / N # Class prior
		D_Ck = Documents in class Ck		
		for w_i in V:			
			# multinomial, bernoulli, gaussian
			p(w_i|Ck) = count_likelihood(...)
	return V, p(Ck), p(w_i|Ck)
```

Алгоритмическая сложность: $O(|D| \langle |\mathbf{x}| \rangle + |C||V|)$

## Применение MultinomialNB


```
function nb_apply(d, C, V, p(Ck), p(w_i|Ck)):
	x = tokenize(d) # somehow	
	for Ck in C: # iterate over all classes
		score(Ck|x) = log p(Ck) # use class prior
		# use likelihoods
		for i in 1..|x|:		
			score(Ck|x) += log p(x_i|Ck)
	return arg max score(Ck|x)
```

Алгоритмическая сложность: $O(|C||\mathbf{x}|)$


## Задача

|d | Текст | Класс |
|--|--|--|
|1 | котики такие мокрые | мимими |
|2 | пушистые котики няшки | мимими |
|3 | морские котики  | не мимими |
|4 | мокрые морские свинки | не мимими |
|5 | котики мокрые | ???|

С помощью алгоритма MultinomialNB вычислить $p(\text{мимими} | d_5)$

## Сглаживание

Проблема: $p(свинки|мимими) = 0$

Решение:

$$ p(x=w_{s_i}|C_k) = \frac{ T_{w^{s_i}, C_k} + \alpha }{ T_{C_k} + \alpha|V|} $$


если $\alpha \geq 0$ - сглаживание Лапласа, если $0 \leq \alpha \leq 1$ - Лидстоуна

**Упражнение**  
С учетом сглаживания вычислить $$p(пушистые|не мимими), p(пушистые|мимими)$$


## Итоги

**+** (Удивительно) неплохо работает  
**+** Стабилен при смещении выборки  
**+** Оптимальный по производительности  

**-** Наивные предположения  
**-** Требует отбора признаков  


## Вопросы
## Пожалуйста, оставьте отзыв о лекции